In [5]:
import numpy as np
import matplotlib.pyplot as plt

class Hybrid_POS_TSP(object):
    def __init__(self,data,num_pop=200):
        self.num_pop = num_pop  #群体个数   
        self.data = data        #城市坐标
        self.num =len(data)     #城市个数 

        #群体的初始化和路径的初始化         
        self.chrom = np.array([0]*self.num_pop*self.num).reshape(self.num_pop,self.num)
        self.fitness = [0]*self.num_pop
        
        #路径矩阵，函数matrix_dis同遗传算法
        self.matrix_distance = self.matrix_dis()


    def cross_1(self,path,best_path):
        r1 = np.random.randint(self.num)
        r2 = np.random.randint(self.num)
        while r2 == r1:
            r2 = np.random.randint(self.num)
                
        left,right = min(r1,r2),max(r1,r2)  
        cross = best_path[left:right+1]
        for i in range(right-left+1):
            for k in range(self.num):
                if path[k] == cross[i]:
                    path[k:self.num-1] = path[k+1:self.num]
                    path[-1] = 0
        path[self.num-right+left-1:self.num] = cross
        return path

    def mutation(self,path):
        r1 = np.random.randint(self.num)
        r2 = np.random.randint(self.num)
        while r2 == r1:
            r2 = np.random.randint(self.num)
        path[r1],path[r2] = path[r2],path[r1]
        return path


def main(data, max_n = 200, num_pop=200):
    Path_short = Hybrid_POS_TSP(data,num_pop=num_pop) #混合粒子群算法类
    Path_short.rand_chrom() #初始化种群
    
    #初始化路径绘图
    fig,ax = plt.subplots()
    x = data[:,0]
    y = data[:,1]
    ax.scatter(x, y,linewidths=0.1)
    for i,txt in enumerate(range(1,len(data)+1)):
        ax.annotate(txt,(x[i],y[i]))
    res0 = Path_short.chrom[0]
    x0 = x[res0]
    y0 = y[res0]
    for i in range(len(data)-1):
        plt.quiver(x0[i], y0[i],x0[i+1]-x0[i], y0[i+1]-y0[i], color='r', width=0.005,angles='xy',scale=1, scale_units='xy')  
    plt.quiver(x0[-1], y0[-1],x0[0]-x0[-1], y0[0]-y0[-1], color='r', width=0.005,angles='xy',scale=1, scale_units='xy')
    plt.show()
    print('初始染色体的路程: '+str( Path_short.fitness[0]))


    #存储个体极值的路径和距离
    best_P_chrom = Path_short.chrom.copy()
    best_P_fit = Path_short.fitness.copy()
    
    min_index = np.argmin(Path_short.fitness)
    
    #存储当前种群极值的路径和距离
    best_G_chrom = Path_short.chrom[min_index,:]
    best_G_fit = Path_short.fitness[min_index]

    #存储每一步迭代后的最优路径和距离
    best_chrom = [best_G_chrom]
    best_fit = [best_G_fit]
    
    #复制当前群体进行交叉变异
    x_new = Path_short.chrom.copy()

    for i in range(max_n):
        #更新当前的个体极值
        for j in range(num_pop):
            if Path_short.fitness[j] < best_P_fit[j]:
                best_P_fit[j] = Path_short.fitness[j]
                best_P_chrom[j,:] = Path_short.chrom[j,:]

        #更新当前种群的群体极值
        min_index = np.argmin(Path_short.fitness)
        best_G_chrom = Path_short.chrom[min_index,:]
        best_G_fit = Path_short.fitness[min_index]
        
        #更新每一步迭代后的全局最优路径和解
        if best_G_fit <best_fit[-1]:
            best_fit.append(best_G_fit)
            best_chrom.append(best_G_chrom)
        else:
            best_fit.append(best_fit[-1])
            best_chrom.append(best_chrom[-1])
        
        #将每个个体与个体极值和当前的群体极值进行交叉
        for j in range(num_pop):
            #与个体极值交叉
            x_new[j,:] = Path_short.cross_1(x_new[j,:], best_P_chrom[j,:])   
            fit = Path_short.comp_fit(x_new[j,:])
            #判断是否保留
            if fit < Path_short.fitness[j]:
                Path_short.chrom[j,:] = x_new[j,:]
                Path_short.fitness[j] = fit
            #与当前极值交叉            
            x_new[j,:] = Path_short.cross_1(x_new[j,:], best_G_chrom)
            fit = Path_short.comp_fit(x_new[j,:])
            if fit < Path_short.fitness[j]:
                Path_short.chrom[j,:] = x_new[j,:]
                Path_short.fitness[j] = fit
            #变异
            x_new[j,:] = Path_short.mutation(x_new[j,:])
            fit = Path_short.comp_fit(x_new[j,:])   
            if fit <= Path_short.fitness[j]:
                Path_short.chrom[j] = x_new[j,:]
                Path_short.fitness[j] = fit
                
        if (i+1)%20 == 0:
            print('第'+str(i+1)+'步后的最短的路程: '+str( Path_short.fitness[min_index]))
            print('第'+str(i+1)+'步后的最优路径:')
            Path_short.out_path(Path_short.chrom[min_index,:])# 显示每一步的最优路径

    Path_short.best_chrom = best_chrom
    Path_short.best_fit = best_fit
    return Path_short       #返回结果类

In [6]:
data = np.array([16.47,96.10,16.47,94.44,20.09,92.54,
    22.39,93.37,25.23,97.24,22.00,96.05,20.47,97.02,
    17.20,96.29,16.30,97.38,14.05,98.12,16.53,97.38,
    21.52,95.59,19.41,97.13,20.09,92.55]).reshape(14,2)

In [7]:
main(data)

AttributeError: 'Hybrid_POS_TSP' object has no attribute 'matrix_dis'